In [1]:
!pip install dash pandas requests plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [8]:
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, html, dcc
import requests
from io import StringIO
from datetime import datetime, timedelta

app = Dash(__name__)

# Define janela de 10 anos
hoje = datetime.today()
dez_anos_atras = hoje - timedelta(days=365 * 10)
data_inicial = dez_anos_atras.strftime('%d/%m/%Y')
data_final = hoje.strftime('%d/%m/%Y')

# URLs com dataInicial e dataFinal para séries diárias (Selic e Câmbio)
url_ipca = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=csv'
url_selic = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.1178/dados?formato=csv&dataInicial={data_inicial}&dataFinal={data_final}'
url_cambio = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?formato=csv&dataInicial={data_inicial}&dataFinal={data_final}'

def carregar_dados(url, nome_coluna):
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Erro ao buscar dados de {url}")
        print(f"Código de status: {response.status_code}")
        print(f"Resposta: {response.text[:500]}")
        raise ConnectionError(f"Erro na URL {url}")

    df = pd.read_csv(StringIO(response.text), sep=';')
    df.columns = df.columns.str.strip().str.lower()

    if 'data' not in df.columns or 'valor' not in df.columns:
        raise KeyError(f"Colunas esperadas não encontradas em {url}")

    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df['valor'] = df['valor'].str.replace(',', '.').astype(float)
    df = df.rename(columns={'valor': nome_coluna})
    return df[['data', nome_coluna]]

# Carrega os dados
df_ipca = carregar_dados(url_ipca, 'ipca')
df_selic = carregar_dados(url_selic, 'selic')
df_cambio = carregar_dados(url_cambio, 'cambio')

# Une os dados
df = df_ipca.merge(df_selic, on='data', how='inner').merge(df_cambio, on='data', how='inner')

# Geração do gráfico
def gerar_grafico():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['data'], y=df['ipca'], mode='lines', name='IPCA (%)'))
    fig.add_trace(go.Scatter(x=df['data'], y=df['selic'], mode='lines', name='Selic (%)'))
    fig.add_trace(go.Scatter(x=df['data'], y=df['cambio'], mode='lines', name='Câmbio (USD/BRL)'))
    fig.update_layout(title='Correlação: Juros, Inflação e Câmbio',
                      xaxis_title='Data', yaxis_title='Valor',
                      hovermode='x unified')
    return fig

# Layout
app.layout = html.Div([
    html.H1('Análise Econômica Interativa'),
    dcc.Graph(figure=gerar_grafico())
])

if __name__ == '__main__':
    app.run(debug=True)





<IPython.core.display.Javascript object>